In [1]:
from compas_notebook.viewer import Viewer
from compas.geometry import Point, Line, Polyline, Vector
from compas.colors import Color
from compas.geometry import Vector
from compas.datastructures import Mesh
import math
import numpy as np

viewer = Viewer()
viewer.scene.clear()

參數設定

In [2]:
R = 3                        # 螺旋半徑
pitch = 40                   # 每一圈上升高度（節距）
turns = 0.25                 # 總圈數
points_per_turn = 200        # 每圈取樣點（越大越平滑）
rungs_per_turn = 100         # 每圈畫幾根「鹼基配對」橫桿
RUNG_WIDTH = 3               # 控制橫桿面的寬度
flor= pitch / rungs_per_turn # 樓層高度
WIND_DIR = Vector(1, 0, 0)   #風向
U = 1.0                      #風速
R_OBS = R*1.15               #障礙物等效半徑
ALPHA = 1.8                  #繞流強度

# 密度與範圍（seeds on YZ plan）
N_Y, N_Z = 2, 3              # 流線密度（越大越密）
Y_RANGE  = (-2.2*R, 2.2*R)
Z_RANGE  = (0.0, pitch*turns)# 在整段 DNA 高度內產生箭頭

# 線段控制
STREAM_LEN   = 6.0 * R         # 每條流線的目標長度
STEP_SIZE    = 0.08 * R        # 數值積分步距（越小越平滑）
SEG_LEN      = 0.20 * R        # 每段線段長
GAP_LEN      = 0.08 * R        # 段與段之間的間隙
HEAD_SCALE   = 0.25            # 箭頭比例（相對於段長）
COLOR_LINE   = Color(0.2, 0.6, 1.0, 0.9)
COLOR_HEAD   = Color(0.2, 0.6, 1.0, 0.9)

畫出DNA雙股

In [3]:
def helix_points(radius, pitch, turns, n_per_turn, phase=0.0):
    pts = []
    total = int(n_per_turn * turns)
    for i in range(total + 1):
        t = 2.0 * math.pi * i / n_per_turn  
        x = radius * math.cos(t + phase)
        y = radius * math.sin(t + phase)
        z = pitch * i / n_per_turn
        pts.append(Point(x, y, z))
    return pts

pts1 = helix_points(R, pitch, turns, points_per_turn, phase=0.0)
pts2 = helix_points(R, pitch, turns, points_per_turn, phase=math.pi)

poly1 = Polyline(pts1)
poly2 = Polyline(pts2)
#viewer.scene.add(poly1, name="strand_A", color=Color(0.15, 0.45, 1.0))  # blue
#viewer.scene.add(poly2, name="strand_B", color=Color(1.0, 0.25, 0.25))  # red

畫出鹼基

In [4]:
step = max(1, int(points_per_turn / rungs_per_turn))   # 每隔多少取樣點畫一根
for i in range(0, min(len(pts1), len(pts2)) - 1, step):
    p = pts1[i]
    q = pts2[i]
    #col = Color(0.75, 0.75, 0.75) if (i // step) % 2 == 0 else Color(0.55, 0.55, 0.55)
    #viewer.scene.add(Line(p, q), color=col)

將每個橫向鹼基變成樓層面

In [5]:
extrude_height = flor

N = int(round(rungs_per_turn * turns))

def quad_at_center_and_dir(center: Point, u_dir: Vector,length: float , width: float):
    u = u_dir.copy()
    if u.length == 0:
        u = Vector(1, 0, 0)
    u.unitize()
    v = Vector(0, 0, 1).cross(u)
    if v.length == 0:
        v = Vector(1, 0, 0).cross(u)
    v.unitize()

    halfL, halfW = 0.5*length, 0.5*width
    a = center + (u * halfL) + (v * halfW)
    b = center - (u * halfL) + (v * halfW)
    c = center - (u * halfL) - (v * halfW)
    d = center + (u * halfL) - (v * halfW)
    return [a, b, c, d]

def prism_from_quad_and_height(quad, height):
    lift = Vector(0, 0, height)
    a, b, c, d = quad
    a2, b2, c2, d2 = a + lift, b + lift, c + lift, d + lift
    vertices = [a, b, c, d, a2, b2, c2, d2]
    faces = [
        [0, 1, 2, 3],  # bottom
        [4, 5, 6, 7],  # top
        [0, 1, 5, 4],
        [1, 2, 6, 5],
        [2, 3, 7, 6],
        [3, 0, 4, 7],
    ]
    return Mesh.from_vertices_and_faces(vertices, faces)

for n in range(N):
    t = 2.0 * math.pi * (n / rungs_per_turn)
    zc = n * flor

    p1 = Point(R * math.cos(t), R * math.sin(t), zc)
    p2 = Point(-R * math.cos(t), -R * math.sin(t), zc)
    center = Point(0, 0, zc)
    u_dir = Vector.from_start_end(p1, p2)
    length = (p1 - p2).length
    quad = quad_at_center_and_dir(center, u_dir, length, RUNG_WIDTH)
    solid = prism_from_quad_and_height(quad, extrude_height)
    
    col = Color(0.80, 0.80, 0.85) if (n % 2 == 0) else Color(0.62, 0.62, 0.68)
    viewer.scene.add(solid)

CFD模擬

In [6]:
def velocity_field(p: Point) -> Vector:
    v = WIND_DIR.unitized()

    # 以 DNA 高度中線為中心製造繞流偏轉（只在 YZ 平面扭曲）
    cy = 0.5 * (Y_RANGE[0] + Y_RANGE[1])
    cz = 0.5 * (Z_RANGE[0] + Z_RANGE[1])
    dy, dz = p.y - cy, p.z - cz
    r2 = dy*dy + dz*dz
    if r2 < (3.0 * R_OBS)**2:
        phi = math.atan2(dz, dy)
        t_y, t_z = -math.sin(phi), math.cos(phi)
        w = ALPHA * (R_OBS**2 / max(r2, R_OBS**2))
        if r2 < (R_OBS**2):
            w *= 2.5
        v.y += w * t_y
        v.z += w * t_z

    if v.length < 1e-9:
        v = Vector(1, 0, 0)

    # 保持在水平層移除 Z 分量(減低電腦運算時間)
    v.z = 0.0

    return v.unitized()

def rk2(p: Point, h: float) -> Point:
    """2 階 Runge-Kutta 積分一步（較平滑）。"""
    k1 = velocity_field(p)
    mid = p + k1 * (0.5 * h)
    k2 = velocity_field(mid)
    return p + k2 * h

MAX_OBJECTS = 8000
_obj_count = 0

def add_arrow_segment(p0: Point, p1: Point):
    """畫一段線段 + 箭頭（小 'V' 形）"""
    global _obj_count
    if _obj_count >= MAX_OBJECTS:
        return
    # 主段
    viewer.scene.add(Line(p0, p1), color=COLOR_LINE);_obj_count += 1

    # 箭頭（末端）
    dirv = Vector.from_start_end(p0, p1)
    L = dirv.length
    if L < 1e-9:
        return
    u = dirv.unitized()
    v = Vector(0, 0, 1).cross(u)
    if v.length < 1e-9:
        v = Vector(0, 1, 0).cross(u)
    v.unitize()

    head_len = HEAD_SCALE * L
    head_wid = 0.5 * HEAD_SCALE * L
    tip  = p1
    base = p1 - u * head_len
    left = base + v * head_wid
    right= base - v * head_wid

    viewer.scene.add(Line(left, tip),  color=COLOR_HEAD)
    viewer.scene.add(Line(right, tip), color=COLOR_HEAD)

def add_streamline(seed: Point):
    """從 seed 沿 +X 積分，並用分段（SEG_LEN / GAP_LEN）繪製箭頭虛線。"""
    total = 0.0
    cur = seed
    carry = 0.0  
    seg_start = cur

    while total < STREAM_LEN:
        nxt = rk2(cur, STEP_SIZE)
        step_d = Vector.from_start_end(cur, nxt).length

        # 碰到「柱體保護圈」就跳過（避免穿體）
        if (cur.y**2 + (cur.z - 0.5*(Z_RANGE[0]+Z_RANGE[1]))**2) < (0.9*R_OBS)**2:
            # 往切線向推一點再繼續
            phi = math.atan2(cur.z - 0.5*(Z_RANGE[0]+Z_RANGE[1]), cur.y)
            push = Vector(-math.sin(phi), math.cos(phi), 0.0) * (0.05*R)
            cur = cur + push
            continue

        carry += step_d
        total += step_d
        cur = nxt

        if carry >= SEG_LEN:
            add_arrow_segment(seg_start, cur)
            dirv = Vector.from_start_end(seg_start, cur).unitized()
            gap_advance = dirv * GAP_LEN
            seg_start = cur + gap_advance
            cur = seg_start
            carry = 0.0

def add_windfield():
    y0, y1 = Y_RANGE
    z0, z1 = Z_RANGE
    for iy in range(N_Y):
        y = y0 + (y1 - y0) * (iy + 0.5) / N_Y
        for iz in range(N_Z):
            z = z0 + (z1 - z0) * (iz + 0.5) / N_Z
            seed = Point(-2.5*R, y, z)  # 從 DNA 左側（負 X）邊界吹入
            add_streamline(seed)

add_windfield()
print("✅ Windfield added successfully.")

✅ Windfield added successfully.


加入圖資說明

In [7]:
def add_info_plate_manual(
    viewer,
    lines,
    anchor="bottom_left",
    z_height=0.0,
    pad=(1.0, 0.6),
    line_spacing=0.6,
    title_size=0.9,
    text_size=0.55,
    plate_width=14.0,
):
    # 1. 定位錨點
    if anchor in ("bottom_left", "top_left"):
        x0 = -plate_width - pad[0]
    else:
        x0 = +pad[0]

    total_lines = len(lines)
    plate_height = (title_size + (total_lines - 1) * line_spacing) + 2 * pad[1]

    if anchor in ("bottom_left", "bottom_right"):
        y0 = -plate_height - pad[1]
    else:
        y0 = +pad[1]

    # 2. 畫出面板與邊框
    from compas.geometry import Point, Vector, Line, Polygon
    from compas.colors import Color

    a = Point(x0, y0, z_height)
    b = Point(x0 + plate_width, y0, z_height)
    c = Point(x0 + plate_width, y0 + plate_height, z_height)
    d = Point(x0, y0 + plate_height, z_height)

    plate = Polygon([a, b, c, d])
    viewer.scene.add(plate, color=Color(1, 1, 1, 0.7))

    for s, t in [(a, b), (b, c), (c, d), (d, a)]:
        viewer.scene.add(Line(s, t), color=Color(0, 0, 0))

    # 3. 加入文字
    y = d.y - pad[1] - title_size * 0.2
    for i, text in enumerate(lines):
        size = title_size if i == 0 else text_size
        pos = Point(a.x + pad[0], y, z_height)
        try:
            viewer.scene.add_text(text, position=pos, height=size, color=Color(0, 0, 0))
        except Exception:
            viewer.scene.add(Line(pos, pos + Vector(plate_width * 0.8, 0, 0)), color=Color(0, 0, 0))
        y += line_spacing
    
    add_info_plate_manual(
    viewer,
    lines=[
        "風洞分析測試",
        "WIND DIRECTION： +X",
        "WIND SPEED： 3.0 m/s",
        "SCALE： 1:150",
        "LOCATION： Taipei, TW",
    ],
    anchor="bottom_right",
    z_height=0.0,
    title_size=0.9,
    text_size=0.55,
)

In [8]:
viewer.show()

In [10]:
import json
import os

data = {
    "brand": "Ford",
    "model": "Mustang",
    "year": 2025
}

# 輸出路徑（用 getcwd 比 __file__ 安全）
output_path = os.path.join(
    os.getcwd(),  # 現在工作資料夾
    'wind_dirc_result.json'   # 檔名
)

# 寫入 JSON
with open(output_path, 'w', encoding='utf-8') as fp:
    json.dump(data, fp, indent=4)
    print(f"✅ JSON 檔案已建立：{output_path}")


✅ JSON 檔案已建立：d:\設計演算\wind_dirc_result.json
